In [1]:
import torch, json, os, random
from torchvision import datasets
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from diffusers import StableDiffusionPipeline
from google.colab import drive

drive.mount('/content/drive')
blip_model_path = "/content/drive/MyDrive/blip_model"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = BlipProcessor.from_pretrained(blip_model_path, local_files_only=True)
blip_model = BlipForConditionalGeneration.from_pretrained(blip_model_path, local_files_only=True).to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Mounted at /content/drive


In [2]:
# cifar
cifar = datasets.CIFAR10(root="data/", train=True, download=True)


100%|██████████| 170M/170M [00:12<00:00, 13.2MB/s]


In [3]:
# Caption modification function
def modify_caption(caption):
    styles = ["high detail", "realistic", "fantasy style", "cartoon smiling"]
    noise = ["banana light chaos", "alien texture burst", "noisy noisy"]
    return {
        "original_caption": caption,
        "style_caption": caption + ", " + random.choice(styles),
        "noisy_caption": caption + ", " + random.choice(noise)
    }

# Step 1: Generate original and modified captions using BLIP
captions = {}
os.makedirs("captions", exist_ok=True)
for idx in tqdm(range(10)):  # 10 for testing
    image = cifar[idx][0].resize((224, 224)).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(device)
    out = blip_model.generate(**inputs)
    raw_caption = processor.decode(out[0], skip_special_tokens=True)
    captions[f"cifar10_{idx}.png"] = modify_caption(raw_caption)

with open("captions/cifar10_captions.json", "w") as f:
    json.dump(captions, f, indent=2)

# Step 2: Generate images with Stable Diffusion
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to(device)
os.makedirs("outputs_sd/cifar10/original", exist_ok=True)
os.makedirs("outputs_sd/cifar10/style", exist_ok=True)
os.makedirs("outputs_sd/cifar10/noisy", exist_ok=True)

for name, data in captions.items():
    # Original caption → SD image
    image_orig = pipe(data["original_caption"]).images[0]
    image_orig.save(os.path.join("outputs_sd/cifar10/original", name))

    # Style-modified caption → SD image
    image_style = pipe(data["style_caption"]).images[0]
    image_style.save(os.path.join("outputs_sd/cifar10/style", name))

    # Noisy caption → SD image
    image_noise = pipe(data["noisy_caption"]).images[0]
    image_noise.save(os.path.join("outputs_sd/cifar10/noisy", name))

# Step 3: Recaption all generated images
regen_captions = {"original": {}, "style": {}, "noisy": {}}

for name in tqdm(captions.keys()):
    for variant in ["original", "style", "noisy"]:
        path = os.path.join(f"outputs_sd/cifar10/{variant}", name)
        image = Image.open(path).resize((224, 224)).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        out = blip_model.generate(**inputs)
        regen_captions[variant][name] = processor.decode(out[0], skip_special_tokens=True)

os.makedirs("regenerated_captions", exist_ok=True)
with open("regenerated_captions/cifar10_sd_captions_extended.json", "w") as f:
    json.dump(regen_captions, f, indent=2)

print("✅ Finished CIFAR-10 original + modified + noisy caption cycle")


100%|██████████| 10/10 [00:03<00:00,  2.92it/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:06<00:00,  1.56it/s]

✅ Finished CIFAR-10 original + modified + noisy caption cycle
